In [1]:
# Dependencies
import requests
import json
from pprint import pprint
import pandas as pd
from pathlib import Path

# Import the API key
from config import geoapify_key

In [4]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
all_zip_data = pd.read_csv(
    Path('zip_codes.csv')   
)

# Review the DataFrame
all_zip_data.head()

,postcode,latitude,longitude
0,3000-vic,-37.814214,144.963145
1,3001-vic,-37.814327,144.962604
2,3002-vic,-37.812872,144.981886
3,3003-vic,-37.809332,144.925223
4,3004-vic,-37.837281,144.972710


In [ ]:
#so because we put the params we dont need to put them into the base URL
# Set the search parameters
countrycode = "au"
params = { 
    "apiKey": geoapify_key,
    "filter": countrycode
}

# Build URL using the geocode endpoint
base_url = "https://api.geoapify.com/v2/places"

# Create a list to store the results
results = []

In [7]:
# base URL
base_url = "https://api.geoapify.com/v1/places"

#parameters for type of establishment 
categories = "healthcare.clinic_or_praxis.general"
bias = f"proximity:{longitude},{latitude}"
limit = 3

# Create an empty DataFrame to store the results
closest_doc = pd.DataFrame(columns=["postcode", "address_line1", "address_line2"])

# Iterate through each postcode in the range from 3000 to 3008 (inclusive)
for postcode in range(3000, 3009):
    # Look up the corresponding latitude and longitude from the all_zip_data DataFrame
    row = all_zip_data[all_zip_data["postcode"] == f"{postcode}-vic"]
    
    if not row.empty:
        latitude = row.iloc[0]["latitude"]
        longitude = row.iloc[0]["longitude"]

    # Make the API request to find the closest doctors
    params = {
        "categories":categories,
        "lat": latitude,
        "lon": longitude,
        "limit":limit,
        "bias":bias,
        "apiKey":geoapify_key 
    }
    

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()

        if data.get("features"):
            # Sort the doctor locations by distance
            doctor_locations = sorted(data["features"], key=lambda x: x["properties"]["distance"])
            
            # Take the top 3 closest doctors (you can adjust the number as needed)
            closest_doctors = doctor_locations[:3]

            for doctor in closest_doctors:
                address_line1 = doctor["properties"]["address_line1"]
                address_line2 = doctor["properties"]["address_line2"]
                
                # Append the data to the result DataFrame
                result_df = result_df.append({
                    "postcode": postcode,
                    "address_line1": address_line1,
                    "address_line2": address_line2
                }, ignore_index=True)
        else:
            print(f"No doctor results found for postcode: {postcode}")
    else:
        print(f"Request failed for postcode: {postcode} with status code: {response.status_code}")

# Print or save the result DataFrame
print(closest_doc)

# If you want to save the results to a CSV file:
# result_df.to_csv("closest_doctors.csv", index=False)


Request failed for postcode: 3000 with status code: 400
Request failed for postcode: 3001 with status code: 400
Request failed for postcode: 3002 with status code: 400
Request failed for postcode: 3003 with status code: 400
Request failed for postcode: 3004 with status code: 400
Request failed for postcode: 3005 with status code: 400
Request failed for postcode: 3006 with status code: 400
Request failed for postcode: 3007 with status code: 400
Request failed for postcode: 3008 with status code: 400
Empty DataFrame
Columns: [postcode, address_line1, address_line2]
Index: []


In [12]:
# Set the geographical coordinates for Brisbane, Australia
latitude = -37.814214 
longitude = 144.963145

# Set the parameters for the type of place
categories = "healthcare.clinic_or_praxis.general"

# Set the parameters for the type of search
bias = f"proximity:{longitude},{latitude}"
limit = 3

# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

In [13]:
# Run request
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
pprint(response)

{'features': [{'geometry': {'coordinates': [144.966178, -37.81360020053079],
                            'type': 'Point'},
               'properties': {'address_line1': 'Swanston Street Medical Centre',
                              'address_line2': '267 Bourke Street, Melbourne '
                                               'VIC 3000, Australia',
                              'categories': ['healthcare',
                                             'healthcare.clinic_or_praxis',
                                             'healthcare.clinic_or_praxis.general'],
                              'city': 'Melbourne',
                              'country': 'Australia',
                              'country_code': 'au',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                             'license': 'Open Database Licence',
                                  